<a href="https://colab.research.google.com/github/inesbsilveira/estufas/blob/main/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0mOo_Ws721Fv2WHm08WmzTEFtKWPkP25sd1gu3wJJxM&tc=x0CphMzHsAsoDgsUy8orspd8HDr0wX_K0WThL2_Pv4Y&cc=IMMx9j0XMTCSDn2oPWQzCGMn0j1p_dgFG13-srhQSnQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnkXLXw3_u3kv3b6OFQ8ew-R_cHbnklR0HFNRFcpjif3h4F5HQ3kBM

Successfully saved authorization token.


In [8]:
import datetime


In [21]:
# Define the region polygon
lonmin = -8.63548
lonmax = -8.34998
latmin = 37.18145
latmax = 37.41591
region = ee.Geometry.Polygon(
    [[lonmin, latmin],
     [lonmax, latmin],
     [lonmax, latmax],
     [lonmin, latmax]])

# Define the Sentinel-2 image collection and filter it by date and region
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(region) \
    .filterDate('2021-01-01', '2021-12-31')  # Change the date range as needed

# Define a function to mask clouds using the QA60 band
def maskClouds(image):
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    qa60 = image.select(['QA60'])
    cloudMask = qa60.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa60.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(cloudMask)

# Map the maskClouds function over the image collection
sentinel2 = sentinel2.map(maskClouds)

# Define a function to add a band containing the date of the image
def addDate(image):
    return image.addBands(image.metadata('system:time_start'))

# Map the addDate function over the image collection
sentinel2 = sentinel2.map(addDate)

# Export each image to Google Drive
image_list = sentinel2.toList(sentinel2.size())
for i in range(0, sentinel2.size().getInfo()):
    image = ee.Image(image_list.get(i))
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    task = ee.batch.Export.image.toDrive(
        image=image.select(['B2', 'B3', 'B4', 'B8']),  # Select the bands you want to export
        description='Sentinel2_' + date.getInfo(),
        folder='Sentinel2_Images',
        region=region,
        scale=10  # Change the scale as needed
    )
    task.start()
